# build vocab
본 노트에서는 자연어처리 관련 딥러닝 모형학습을 위한 필수적인 전처리중 하나인 training corpus에 존재하는 token들의 집합인 **vocabulary**를 만들어 봅니다. vocabulary 구성 자체는 미리 제공하는 `model.utils` module에 있는 `Vocab` class를 활용해봅니다. `model.utils` module에 있는 `Tokenizer` class, `PadSequence` class를 같이 활용하여, 효율적인 전처리를 어떻게 할 수 있는 지 확인합니다.

### Setup

In [1]:
import sys
import pickle
import itertools
import pandas as pd
from pathlib import Path
from pprint import pprint
from typing import List
from collections import Counter
from model.utils import Vocab, Tokenizer, PadSequence

### Load dataset

In [2]:
data_dir = Path.cwd() / 'data'
list_of_dataset = list(data_dir.iterdir())
pprint(list_of_dataset)

[PosixPath('/root/Documents/archive/strnlp/exercise/data/.DS_Store'),
 PosixPath('/root/Documents/archive/strnlp/exercise/data/morphs_eojeol.pkl'),
 PosixPath('/root/Documents/archive/strnlp/exercise/data/train.txt'),
 PosixPath('/root/Documents/archive/strnlp/exercise/data/morphs_vec.pkl'),
 PosixPath('/root/Documents/archive/strnlp/exercise/data/validation.txt'),
 PosixPath('/root/Documents/archive/strnlp/exercise/data/tokenizer.pkl'),
 PosixPath('/root/Documents/archive/strnlp/exercise/data/test.txt'),
 PosixPath('/root/Documents/archive/strnlp/exercise/data/vocab.pkl')]


In [4]:
tr_dataset = pd.read_csv(list_of_dataset[2], sep='\t')
tr_dataset.head()

,document,label
0,애들 욕하지마라 지들은 뭐 그렇게 잘났나? 솔까 거기 나오는 귀여운 애들이 당신들보...,1
1,여전히 반복되고 있는 80년대 한국 멜로 영화의 유치함.,0
2,쉐임리스 스티브와 피오나가 손오공 부르마로 ㅋㅋㅋ,0
3,0점은 없나요?...,0
4,제발 시즌2 ㅜㅜ,1


### Split training corpus and count each tokens
앞선 `eda.ipynb` 노트에서 활용한 `Mecab` class의 instance의 멤버함수인 `morphs` 이용, **training corpus를 sequence of tokens의 형태로 변환하고, training corpus에서 token의 출현빈도를 계산합니다.**

In [5]:
# 문장을 어절기준으로 보는 split_fn을 작성
def split_eojeol(s: str) -> List[str]:
    return s.split(' ')

In [6]:
training_corpus = tr_dataset['document'].apply(lambda sen: split_eojeol(sen)).tolist()
pprint(training_corpus[:5])

[['애들',
  '욕하지마라',
  '지들은',
  '뭐',
  '그렇게',
  '잘났나?',
  '솔까',
  '거기',
  '나오는',
  '귀여운',
  '애들이',
  '당신들보다',
  '훨',
  '낮다.'],
 ['여전히', '반복되고', '있는', '80년대', '한국', '멜로', '영화의', '유치함.'],
 ['쉐임리스', '스티브와', '피오나가', '손오공', '부르마로', 'ㅋㅋㅋ'],
 ['0점은', '없나요?...'],
 ['제발', '시즌2', 'ㅜㅜ']]


In [24]:
count_tokens = Counter(itertools.chain.from_iterable(training_corpus))
print(len(count_tokens))

299101


### Build vocab
`min_freq`를 설정하고, training corpus에서 출현빈도가 `min_freq` 미만인 token을 제외하여, `model.utils` module에 있는 `Vocab` class를 이용하여 Vocabulary를 구축합니다. `min_freq`보다 낮은 token들은 `unknown`으로 처리됩니다. 아래의 순서로 진행합니다.

1. `list_of_tokens`을 만듭니다. `list_of_tokens`은 `min_freq` 이상 출현한 token들을 모아놓은 `list`입니다.
2. `Vocab` class의 instance인 `vocab`을 만듭니다. `list_of_tokens`를 parameter로 전달받습니다.

ps. tutorial에 사용되는 논문은 pretrained word vector를 사용하는 데, 원활한 진행을 위해서 제가 미리 준비해놓은 pretrained wordvector를 사용하도록 하겠습니다.

In [26]:
min_freq = 10

In [28]:
list_of_tokens = [token for token in count_tokens.keys() if count_tokens.get(token) >= min_freq]

In [31]:
list_of_tokens = sorted(list_of_tokens)
print(len(list_of_tokens))

9394


In [37]:
vocab = Vocab(list_of_tokens=list_of_tokens, bos_token=None, eos_token=None, unknown_token_idx=0)

In [ ]:
with open('data/morphs_eojeol.pkl', mode='wb') as io:
#     pickle.dump(array, io)

In [40]:
# tutorial 진행을 위해서 위에서 생성한 vocabulary의 token들의 embedding vector를 가져옵니다.
with open('data/morphs_eojeol.pkl', mode='rb') as io:
    morphs_eojeol = pickle.load(io)
print(morphs_eojeol)

[[ 0.         0.         0.        ...  0.         0.         0.       ]
 [ 0.         0.         0.        ...  0.         0.         0.       ]
 [-0.25759   -0.014235  -1.2187    ... -0.097028   0.48018    0.28739  ]
 ...
 [-0.13822    0.23503   -0.29733   ... -0.29198    0.18319   -0.10823  ]
 [-0.028508   0.47924    0.0081392 ... -0.27024    0.17462    0.28043  ]
 [-0.11004    0.12825   -0.10196   ... -0.14461   -0.085906   0.18802  ]]


In [43]:
vocab.embedding = morphs_eojeol

In [42]:
vocab.to_indices('40대')

183

In [50]:
len(vocab)

9396

In [49]:
vocab.embedding.shape

(9396, 300)

### How to use `Vocab`
`list_of_tokens`로 생성한 `Vocab` class의 instance인 `vocab`은 아래와 같은 멤버들을 가지고 있습니다.

In [51]:
help(Vocab)

Help on class Vocab in module model.utils:

class Vocab(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self, list_of_tokens=None, padding_token='<pad>', unknown_token='<unk>', bos_token='<bos>', eos_token='<eos>', reserved_tokens=None, unknown_token_idx=0)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __len__(self)
 |  
 |  to_indices(self, tokens:Union[str, List[str]]) -> Union[int, List[int]]
 |  
 |  to_tokens(self, indices:Union[int, List[int]]) -> Union[str, List[str]]
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  bos_token
 |  
 |  embedding
 |  
 |  eos_token
 |  
 |  idx_to_token
 |  
 |  padding_token
 |  
 |  token_to_idx
 |  
 |  unknown_token



In [52]:
# padding_token, unknown_token, eos_token, bos_token
print(vocab.padding_token)
print(vocab.unknown_token)
print(vocab.eos_token)
print(vocab.bos_token)

<pad>
<unk>
None
None


In [53]:
# token_to_idx
print(vocab.token_to_idx)

{'<unk>': 0, '<pad>': 1, '!': 2, '!!': 3, '!!!': 4, '!!!!': 5, '""': 6, '"""': 7, '"이': 8, '&': 9, "'": 10, '(10자': 11, ')': 12, '+': 13, ',': 14, ',,': 15, ',,,': 16, '-': 17, '--': 18, '->': 19, '-_': 20, '-_-': 21, '-_-;': 22, '-_-;;': 23, '.': 24, '..': 25, '...': 26, '....': 27, '.....': 28, '......': 29, '/': 30, '//': 31, '0': 32, '007': 33, '0개는': 34, '0점': 35, '0점도': 36, '0점은': 37, '0점을': 38, '0점이': 39, '1': 40, '1,': 41, '1,2': 42, '1.': 43, '10': 44, '10,': 45, '100%': 46, '100배': 47, '100점': 48, '10개': 49, '10년': 50, '10년도': 51, '10년만에': 52, '10년이': 53, '10년전': 54, '10년전에': 55, '10대': 56, '10번': 57, '10번도': 58, '10분': 59, '10자': 60, '10점': 61, '10점!': 62, '10점.': 63, '10점도': 64, '10점만점에': 65, '10점은': 66, '10점을': 67, '10점이': 68, '10점이다.': 69, '10점주는': 70, '10점준': 71, '10점준다': 72, '10점준다.': 73, '10점줌': 74, '10점짜리': 75, '11점을': 76, '12세': 77, '15세': 78, '18': 79, '19금': 80, '1개': 81, '1개도': 82, '1도': 83, '1등': 84, '1시간': 85, '1위': 86, '1은': 87, '1을': 88, '1이': 89, '1인': 90, '1

In [54]:
# idx_to_token
print(vocab.idx_to_token)

{0: '<unk>', 1: '<pad>', 2: '!', 3: '!!', 4: '!!!', 5: '!!!!', 6: '""', 7: '"""', 8: '"이', 9: '&', 10: "'", 11: '(10자', 12: ')', 13: '+', 14: ',', 15: ',,', 16: ',,,', 17: '-', 18: '--', 19: '->', 20: '-_', 21: '-_-', 22: '-_-;', 23: '-_-;;', 24: '.', 25: '..', 26: '...', 27: '....', 28: '.....', 29: '......', 30: '/', 31: '//', 32: '0', 33: '007', 34: '0개는', 35: '0점', 36: '0점도', 37: '0점은', 38: '0점을', 39: '0점이', 40: '1', 41: '1,', 42: '1,2', 43: '1.', 44: '10', 45: '10,', 46: '100%', 47: '100배', 48: '100점', 49: '10개', 50: '10년', 51: '10년도', 52: '10년만에', 53: '10년이', 54: '10년전', 55: '10년전에', 56: '10대', 57: '10번', 58: '10번도', 59: '10분', 60: '10자', 61: '10점', 62: '10점!', 63: '10점.', 64: '10점도', 65: '10점만점에', 66: '10점은', 67: '10점을', 68: '10점이', 69: '10점이다.', 70: '10점주는', 71: '10점준', 72: '10점준다', 73: '10점준다.', 74: '10점줌', 75: '10점짜리', 76: '11점을', 77: '12세', 78: '15세', 79: '18', 80: '19금', 81: '1개', 82: '1개도', 83: '1도', 84: '1등', 85: '1시간', 86: '1위', 87: '1은', 88: '1을', 89: '1이', 90: '1인', 91

In [56]:
# to_indices
example_sentence = tr_dataset['document'][0]
tokenized_sentence = split_eojeol(example_sentence)
transformed_sentence = vocab.to_indices(tokenized_sentence)
print(example_sentence)
print(tokenized_sentence)
print(transformed_sentence)

애들 욕하지마라 지들은 뭐 그렇게 잘났나? 솔까 거기 나오는 귀여운 애들이 당신들보다 훨 낮다.
['애들', '욕하지마라', '지들은', '뭐', '그렇게', '잘났나?', '솔까', '거기', '나오는', '귀여운', '애들이', '당신들보다', '훨', '낮다.']
[5429, 0, 0, 3235, 1150, 0, 4521, 722, 1502, 1063, 5434, 0, 9316, 1666]


In [57]:
# to_tokens
print(vocab.to_tokens(transformed_sentence))

['애들', '<unk>', '<unk>', '뭐', '그렇게', '<unk>', '솔까', '거기', '나오는', '귀여운', '애들이', '<unk>', '훨', '낮다.']


### How to use `Tokenizer`
위의 `Vocab` class의 활용 형태를 보면 `split_fn`으로 활용하는 `split_morphs` function의 결과를 input을 기본적으로 받습니다. `Vocab` class의 instance와 `split_fn`으로 활용하는 `split_morphs` function을 parameter로 전달받아 전처리를 통합적인 형태의 `Tokenizer` class를 활용할 수 있습니다. `composition` 형태를 이용하여 구현합니다.

In [59]:
help(Tokenizer)

Help on class Tokenizer in module model.utils:

class Tokenizer(builtins.object)
 |  Tokenizer class
 |  
 |  Methods defined here:
 |  
 |  __init__(self, vocab:model.utils.Vocab, split_fn:Callable[[str], List[str]], pad_fn:Callable[[List[int]], List[int]]=None) -> None
 |      Instantiating Tokenizer class
 |      
 |      Args:
 |          vocab (model.utils.Vocab): the instance of model.utils.Vocab created from specific split_fn
 |          split_fn (Callable): a function that can act as a splitter
 |          pad_fn (Callable): a function that can act as a padder
 |  
 |  split(self, string:str) -> List[str]
 |  
 |  split_and_transform(self, string:str) -> List[int]
 |  
 |  transform(self, list_of_tokens:List[str]) -> List[int]
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object 

In [61]:
tokenizer = Tokenizer(vocab, split_eojeol)

In [62]:
# split, transform, split_and_transform
example_sentence = tr_dataset['document'][1]
tokenized_sentence = tokenizer.split(example_sentence)
transformed_sentence = tokenizer.transform(tokenized_sentence)
print(example_sentence)
print(tokenized_sentence)
print(transformed_sentence)

여전히 반복되고 있는 80년대 한국 멜로 영화의 유치함.
['여전히', '반복되고', '있는', '80년대', '한국', '멜로', '영화의', '유치함.']
[5833, 0, 7002, 202, 8988, 2949, 6119, 6546]


In [63]:
print(tokenizer.split_and_transform(example_sentence))

[5833, 0, 7002, 202, 8988, 2949, 6119, 6546]


`model.utils` module에 있는 `PadSequence`를 활용, `Tokenizer` class의 instance인 `tokenizer`에 padding 기능을 추가할 수 있습니다. 이 때 padding은 vocabulary에서 `<pad>`가 가리키고 있는 정수값을 활용합니다.

In [64]:
padding_value = vocab.to_indices(vocab.padding_token)
print(padding_value)

1


In [65]:
pad_sequence = PadSequence(length=32, pad_val=padding_value)

In [67]:
tokenizer = Tokenizer(vocab, split_eojeol, pad_sequence)

In [68]:
# split, transform, split_and_transform
example_sentence = tr_dataset['document'][1]
tokenized_sentence = tokenizer.split(example_sentence)
transformed_sentence = tokenizer.transform(tokenized_sentence)
print(example_sentence)
print(tokenized_sentence)
print(transformed_sentence)

여전히 반복되고 있는 80년대 한국 멜로 영화의 유치함.
['여전히', '반복되고', '있는', '80년대', '한국', '멜로', '영화의', '유치함.']
[5833, 0, 7002, 202, 8988, 2949, 6119, 6546, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


### Save `vocab`

In [69]:
with open('data/vocab.pkl', mode='wb') as io:
    pickle.dump(vocab, io)